# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 30 2023 9:02AM,256144,19,9068457,"Snap Medical Industries, LLC",Released
1,Jan 30 2023 9:00AM,256143,10,Eme-108271,Emerginnova,Released
2,Jan 30 2023 9:00AM,256143,10,DNMD-33063,Emerginnova,Released
3,Jan 30 2023 9:00AM,256143,10,DNMD-33072,Emerginnova,Released
4,Jan 30 2023 8:56AM,256142,10,PRF-49283,Bio-PRF,Released
5,Jan 30 2023 8:54AM,256141,10,PRF-49548,Bio-PRF,Released
6,Jan 30 2023 8:54AM,256141,10,PRF-49771,Bio-PRF,Released
7,Jan 30 2023 8:54AM,256141,10,PRF-49821,Bio-PRF,Released
8,Jan 30 2023 8:54AM,256141,10,PRF-50032,Bio-PRF,Released
9,Jan 30 2023 8:54AM,256141,10,9093840,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
8,256140,Released,1
9,256141,Released,7
10,256142,Released,1
11,256143,Released,3
12,256144,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
256140,NaN,1.0
256141,NaN,7.0
256142,NaN,1.0
256143,NaN,3.0
256144,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
256131,9.0,2.0
256132,0.0,1.0
256133,0.0,1.0
256135,0.0,34.0
256137,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
256131,9,2
256132,0,1
256133,0,1
256135,0,34
256137,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,256131,9,2
1,256132,0,1
2,256133,0,1
3,256135,0,34
4,256137,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,256131,9,2
1,256132,,1
2,256133,,1
3,256135,,34
4,256137,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 30 2023 9:02AM,256144,19,"Snap Medical Industries, LLC"
1,Jan 30 2023 9:00AM,256143,10,Emerginnova
4,Jan 30 2023 8:56AM,256142,10,Bio-PRF
5,Jan 30 2023 8:54AM,256141,10,Bio-PRF
12,Jan 30 2023 8:51AM,256140,12,Hush Hush
13,Jan 30 2023 8:51AM,256139,10,ISDIN Corporation
26,Jan 30 2023 8:47AM,256138,19,"Graystone, LLC"
27,Jan 30 2023 8:46AM,256137,19,"Graystone, LLC"
28,Jan 30 2023 8:40AM,256135,12,Hush Hush
62,Jan 30 2023 8:06AM,256133,22,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jan 30 2023 9:02AM,256144,19,"Snap Medical Industries, LLC",,1
1,Jan 30 2023 9:00AM,256143,10,Emerginnova,,3
2,Jan 30 2023 8:56AM,256142,10,Bio-PRF,,1
3,Jan 30 2023 8:54AM,256141,10,Bio-PRF,,7
4,Jan 30 2023 8:51AM,256140,12,Hush Hush,,1
5,Jan 30 2023 8:51AM,256139,10,ISDIN Corporation,,13
6,Jan 30 2023 8:47AM,256138,19,"Graystone, LLC",,1
7,Jan 30 2023 8:46AM,256137,19,"Graystone, LLC",,1
8,Jan 30 2023 8:40AM,256135,12,Hush Hush,,34
9,Jan 30 2023 8:06AM,256133,22,"NBTY Global, Inc.",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 30 2023 9:02AM,256144,19,"Snap Medical Industries, LLC",1,
1,Jan 30 2023 9:00AM,256143,10,Emerginnova,3,
2,Jan 30 2023 8:56AM,256142,10,Bio-PRF,1,
3,Jan 30 2023 8:54AM,256141,10,Bio-PRF,7,
4,Jan 30 2023 8:51AM,256140,12,Hush Hush,1,
5,Jan 30 2023 8:51AM,256139,10,ISDIN Corporation,13,
6,Jan 30 2023 8:47AM,256138,19,"Graystone, LLC",1,
7,Jan 30 2023 8:46AM,256137,19,"Graystone, LLC",1,
8,Jan 30 2023 8:40AM,256135,12,Hush Hush,34,
9,Jan 30 2023 8:06AM,256133,22,"NBTY Global, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 30 2023 9:02AM,256144,19,"Snap Medical Industries, LLC",1,
1,Jan 30 2023 9:00AM,256143,10,Emerginnova,3,
2,Jan 30 2023 8:56AM,256142,10,Bio-PRF,1,
3,Jan 30 2023 8:54AM,256141,10,Bio-PRF,7,
4,Jan 30 2023 8:51AM,256140,12,Hush Hush,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jan 30 2023 9:02AM,256144,19,"Snap Medical Industries, LLC",1,NaN
1,Jan 30 2023 9:00AM,256143,10,Emerginnova,3,NaN
2,Jan 30 2023 8:56AM,256142,10,Bio-PRF,1,NaN
3,Jan 30 2023 8:54AM,256141,10,Bio-PRF,7,NaN
4,Jan 30 2023 8:51AM,256140,12,Hush Hush,1,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 30 2023 9:02AM,256144,19,"Snap Medical Industries, LLC",1,0.0
1,Jan 30 2023 9:00AM,256143,10,Emerginnova,3,0.0
2,Jan 30 2023 8:56AM,256142,10,Bio-PRF,1,0.0
3,Jan 30 2023 8:54AM,256141,10,Bio-PRF,7,0.0
4,Jan 30 2023 8:51AM,256140,12,Hush Hush,1,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1024565,24,0.0
12,512275,35,0.0
19,1280682,6,9.0
22,256133,1,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1024565,24,0.0
1,12,512275,35,0.0
2,19,1280682,6,9.0
3,22,256133,1,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,24,0.0
1,12,35,0.0
2,19,6,9.0
3,22,1,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,24.0
1,12,Released,35.0
2,19,Released,6.0
3,22,Released,1.0
4,10,Executing,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,19,22
Status,,,,
Executing,0.0,0.0,9.0,0.0
Released,24.0,35.0,6.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,19,22
0,Executing,0.0,0.0,9.0,0.0
1,Released,24.0,35.0,6.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,19,22
0,Executing,0.0,0.0,9.0,0.0
1,Released,24.0,35.0,6.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()